![Python](assets/python-logo-generic.svg)
### Useful packages
#### 김응섭

## Outline
* PIP
* Pattern matching
* Command line 처리
* File 찾기
* Subprocess

## PyPI
* PyPI is the default Package Index for the Python community
* http://pypi.python.org
* How to install a PyPI package:

## How to install a PyPI package:
* 집에서

```bash
$ pip install <package_name>
```

* 회사 PC

```bash
$ pip —-proxy http://12.26.262.2:8080 —-trusted-host pypi.python.org
```

## How about VWP?
1. PyPI에서 원하는 패키지 이름을 검색
1. 저한테 이메일로 패키지 이름을 알려주세요
3. 설치는 그 주 주말에 이루어질 거에요

## Pattern matching
* Regular expression (re)
* PyParsing

## regular expression
* 문자열 매칭을 위한 패턴을 기술하는 방법
* Python은 built-in module, re를 통해 지원

## Re example

In [1]:
import re

uvm_message = r"UVM_(?P<umsg_severity>(ERROR|FATAL))\s+\((?P<reporter>.*)\)\s*(?P<umsg>.*)"

def test_uvm_message():
    test_str = """[805] UVM_ERROR (sec_apb0_reg_adp) UVM_READ, addr = 'h0000000000000408, data = 'h00000000"""
    expect = {'umsg_severity': 'ERROR',
              'reporter': 'sec_apb0_reg_adp',
              'umsg': "UVM_READ, addr = 'h0000000000000408, data = 'h00000000"}
    found = re.search(uvm_message, test_str)
    for key, val in expect.items():
        print("{}: {}".format(key, val))
        assert found.group(key) == expect[key]
test_uvm_message()

reporter: sec_apb0_reg_adp
umsg_severity: ERROR
umsg: UVM_READ, addr = 'h0000000000000408, data = 'h00000000


## Command line 처리
* Command line
* argv
* optparse

## Command line 




* Script 사용자를 위한 인터페이스
* Shell에서 script 실행시에 사용
* example

```bash
$ my_script.py -o <optarg> -i <arg0> <arg1> ...
```

## Posix guideline
    * An option is a hyphen followed by a single alphanumeric character, like this: -o.
    * An option may require an argument (which must appear immediately after the option); for * example, -o argument or -oargument.
    * Options that do not require arguments can be grouped after a hyphen, so, for example, -lst * is equivalent to -t -l -s.
    * Options can appear in any order; thus -lst is equivalent to -tls.
    * Options can appear multiple times.
    * Options precede other nonoption arguments: -lst nonoption.
    * The -- argument terminates options.
    * The - option is typically used to represent one of the standard input streams.

## GNU guideline
* POSIX guideline + long option
* Long options consist of ‘--’ followed by a name made of alphanumeric characters and dashes.
* To specify an argument for a long option, write ‘--name=value’.
    * But, many utils use like `--name value`, instead

## EDA tool convention
* Long option with '-' like -version
    * can't group options: `-t -l -s` -> `-tls`
* `+` argumen: user-specific options

## Command line args

* sys.argv: args split by white spaces

```python
#!/usr/bin/env python
import sys

for e in sys.argv:
    print(e)
```

```bash
$ ./cmd.py -o out_file -i --long-option=long-value --long opt arg0 arg1
./cmd.py
-o
out_file
-i
--long-option=long-value
--long
opt
arg0
arg1
```

## Command line parsers
* getopt
    * C-style parser for command line options
* optparse
    * Only GNU style command line options
    * Deprecated since version 3.2
* argparse
    * Present standard
    * GNU style, EDA style, Sub commands

## argparse example

```python
import argparse

parser = argparse.ArgumentParser(description='Process some integers.')
parser.add_argument('integers', metavar='N', type=int, nargs='+',
                    help='an integer for the accumulator')
parser.add_argument('--sum', dest='accumulate', action='store_const',
                    const=sum, default=max,
                    help='sum the integers (default: find the max)')

args = parser.parse_args()
print(args.accumulate(args.integers))
```

```bash
$ python prog.py -h
usage: prog.py [-h] [--sum] N [N ...]

Process some integers.

positional arguments:
 N           an integer for the accumulator

optional arguments:
 -h, --help  show this help message and exit
 --sum       sum the integers (default: find the max)
```

## File 찾기
* os.path
* pathlib

## os.path
* Path를 나타내는 방식이 OS마다 다름
    * Unix: `/usr/local/bin`
    * Windows: `C:\python64`
* os.path
    * OS-independent file operation 제공

## os.path operations
* os.sep: path 구분 문자
* os.getcwd(): 현재 디렉토리를 리턴
* os.path.join()

In [2]:
import os
print(os.sep)
print(os.path.join(os.getcwd(), 'x.html'))

/
/home/nbuser/library/x.html


## os.path operations
* os.path.abspath(f)
* os.path.split(f): (dir, file)
* os.path.splitext(f): (path, ext)
* os.path.dirname(f): dir
* os.path.basename(f): filename
* os.path.relpath(f, start_path)

## os.path operations

In [3]:
import os
f = os.path.join(os.getcwd(), 'x.html')
assert os.path.abspath('x.html') == f
print(os.path.split(f))
print(os.path.splitext(f))
print(os.path.dirname(f))
print(os.path.basename(f))
print(os.path.relpath(f, os.getcwd()))

('/home/nbuser/library', 'x.html')
('/home/nbuser/library/x', '.html')
/home/nbuser/library
x.html
x.html


## os.path operations
* Test functions

In [4]:
import os
wd = os.getcwd()
print(os.path.exists(wd))
print(os.path.isdir(wd))
print(os.path.isfile(wd))
print(os.path.islink(wd))

True
True
False
False


## pathlib
* python에서 도입된 고수준 path 조작 라이브러리
* Object-oriented filesystem paths
    * os.path는 str을 다루지만, pathlib은 Path obj를 다룸

## pathlib example
* Listing subdirectories

In [5]:
from pathlib import Path

p = Path('.')
print([x for x in p.iterdir() if x.is_dir()])

[PosixPath('.git'), PosixPath('.ipynb_checkpoints'), PosixPath('assets')]


## pathlib example
* Recursive하게 하위 디렉토리 검색
    * `*`: 현재 디렉토리 검색
    * `**`: 모든 하위디렉토리 검색

In [6]:
from pathlib import Path

for p in Path('.').glob('**/*.png'):
    print(p)

assets/python-logo-master-v3-TM.png
assets/opengraph-icon-200x200.png
assets/python.png


## pathlib operators
* '/': join
* str(): Path obj -> str

In [7]:
from pathlib import Path

p = Path('.')
p1 = p / 'x.html'
assert not isinstance(p1, str)
print(str(p1))

x.html


## pathlib attributes
* parts
* parent
* name
* suffix
* stem

## pathlib methods
* match
* relative_to
* with_name
* with_suffix

## pathlib methods
* p.exists()
* p.is_dir()
* p.is_file()
* p.is_symlink()

## pathlib static methods
* Path.cwd()
* Path.home()

## pathlib: file/dir 조작
* mkdir
* open
* rename
* unlink
* rmdir

## Subprocess
* subprocess
* sh

## Subprocess
* Python에서 program을 실행시키기 위한 방법을 담은 standard library
* call(): 명령 실행 후 exit code 리턴

In [12]:
from subprocess import call

sts = call('date')
print('Return:', sts)

Return: 0


## Subprocess
* check_output(): stdout을 리턴

In [13]:
from subprocess import check_output

output = check_output('date')
print('output:', output.decode('utf-8'))

output: Fri Jun 29 08:19:22 UTC 2018



## Subprocess
* Popen()
    * PIPE 처리 등, 정밀 처리를 위해 사용
* communicate()
    * process의 I/O를 담당
    * Process가 종료되기를 기다림

In [21]:
from subprocess import Popen, PIPE

p = Popen('echo', stdin=PIPE, encoding='utf-8')
p.communicate('Hello, world')

TypeError: __init__() got an unexpected keyword argument 'encoding'

## Popen
* subprocess.PIPE
    * IO를 Python에서 제어하도록 지정
* (stdout, stderr) = communicate()
    * process의 출력 내용을 tuple로 돌려줌

In [22]:
from subprocess import Popen, PIPE

p = Popen(['ls', '-al', 'old'], encoding='utf-8',
            stdout=PIPE, stderr=PIPE)
(out, err) = p.communicate()
print('out:', out)
print('err:', err)

TypeError: __init__() got an unexpected keyword argument 'encoding'

## PIPE example
* `ls | sort`

In [23]:
from subprocess import Popen, PIPE

p_sort = Popen(['sort'], stdin=PIPE, stdout=PIPE, encoding='utf-8')
p_ls = Popen(['ls', 'old'], stdout=p_sort.stdin, stdin=PIPE, encoding='utf-8')
result = p_sort.communicate()
print(result[0])

TypeError: __init__() got an unexpected keyword argument 'encoding'

## os.environ
* Subprocess 실행 시에 environment variable을 변경 가능

In [25]:
import os
from subprocess import Popen
new_env = os.environ.copy()
new_env['VAR'] = 'hello'
p_sort = Popen("echo var: $VAR", shell=True, encoding='utf-8')
p_sort.communicate()

TypeError: __init__() got an unexpected keyword argument 'encoding'

## sh
* Subprocess를 쉽게 제어할 수 있게 제작된 3rd-party library
* https://amoffat.github.io/sh

```python
import sh
sh.ls('-l')
```

## TODO
- PIP 설명 변경 필요
- Regular expression에 대한 설명 추가
- pathlib에 실전 예제 추가
- subprocess는 제외